In [113]:
import scipy.stats as stats
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression, Lasso
import scipy
import statsmodels.api as sm
from statsmodels.stats.anova import anova_lm
import io   
import matplotlib.pyplot as plt
from scipy.optimize import linprog



In [2]:
# from chatgpt
def generate_orthonormal_matrix(m, n):
    # Generate a random matrix
    A = np.random.rand(m, n)

    # Perform SVD decomposition
    U, _, Vt = np.linalg.svd(A, full_matrices=False)

    # Ensure that U and Vt have positive determinant
    # print(U.shape, Vt.shape)
    # U *= np.linalg.det(U)
    # V = Vt.T * np.linalg.det(Vt)
    
    # Construct orthonormal matrix
    Q = np.matmul(U, Vt)

    return Q


def generate_data(size=(1000,950), orthonormal=False, sigma=1):
    if orthonormal:
        X = generate_orthonormal_matrix(size[0], size[1])
    else:
        X = np.random.normal(loc=0.0, scale=sigma, size=size)
    beta = np.zeros((size[1],1))
    beta[:5] = 3
    eps = np.random.normal(size=(size[0],1))
    
    Y = X @ beta + eps
    return X, Y, beta, eps


X, Y, beta, eps = generate_data(orthonormal=True)

def process_summary(model_summary):
    table_summary = model_summary.tables[1].as_csv()
    df = pd.read_csv(io.StringIO(table_summary), sep=",")
    df = df.rename(columns={i:i.strip() for i in list(df.columns[1:])})
    return df


In [3]:
def task1(size, k=5, sigma=1, repetition=5):

    beta_true_arr = np.zeros((repetition, size[1]))
    beta_ridge_arr = np.zeros((repetition, size[1]))
    beta_ols_arr = np.zeros((repetition, size[1]))


    n_rows, n_cols = size
    for i in range(repetition):
        
        X = generate_orthonormal_matrix(size[0], size[1])
        beta = np.zeros((n_cols,1))
        beta[:k] = 3.5
        
        gamma_0 = n_cols * sigma**2 / (np.sum(beta**2)) # 3.7
        # print(beta.shape
        beta_true_arr[i] = np.squeeze(beta)
        eps = np.random.normal(size=(n_rows,1))
        Y = X @ beta + eps

        # i) minimize MSE and compute lambda
        # computed on the blackboard,
        # ii)
        # formula from the lecture
        
        beta_ridge = np.linalg.inv(X.T@X + gamma_0 * np.eye(n_cols)) @  X.T @ Y

        # MSE = np.sum((beta - beta_ridge)**2) / n_rows # by a model # important, consider only first k features
        
        #bias - avg error within one coordinate
        # bias = np.sum(beta - beta_ridge)
        #variance - change to one coordinate within beta after many iteration
        # var = np.std(beta_est) ** 2
        beta_ridge_arr[i] = np.squeeze(beta_ridge)

        # OLS part
        # print(X.shape, Y.shape)
        # model = lin_reg_task_a(X, Y, col_num=k)
        model = sm.OLS(Y,X).fit(alpha=0.1)
        df = process_summary(model.summary())
        beta_ols = np.array(df['coef'])
        # print(beta_ols[:5])
        beta_ols_arr[i] = beta_ols

        # print(beta_est)

    # after experiments compute
    def compute_stats(beta_true, beta_est):
        # mse
        mse = np.sum((beta_true - beta_est)**2, axis=1) / beta_true.shape[1]
        # bias
        bias = np.abs(np.mean(beta_true - beta_est, axis=0))
        # variance
        var = np.std(beta_true - beta_est, axis=0) ** 2

        return mse, bias, var

    def theoretical_stats_ridge(beta_true, sigma=1, n_col=950):
        beta_sum_sqr = np.sum(beta_true**2)
        var = sigma**2 / (1 + n_cols * sigma**2 / (np.sum(beta_true[:k]**2)))**2
        mse = n_col * sigma**2 * beta_sum_sqr / (n_col * sigma**2  + beta_sum_sqr)
        bias = beta_true * (-sigma**2 * n_col) / (beta_sum_sqr + sigma**2 * n_col)
        return mse, bias, var

    # MSE_ridge = np.sum((beta_true_arr - beta_ridge_arr)**2, axis=1) / n_rows
    # print(MSE_ridge)

    # MSE_ols = np.sum((beta_true_arr - beta_ols_arr)**2, axis=1) / n_rows
    # print(MSE_ols)

    # bias - avg per coordinate, different for significant and not significant

    # variance - again avg per coordinate
    stats_ridge = compute_stats(beta_true_arr, beta_ridge_arr)

    stats_ols = compute_stats(beta_true_arr, beta_ols_arr)
    print(beta_true_arr[0][:25])
    stats_theoretical = theoretical_stats_ridge(beta_true_arr[0])

    def print_some_stats(stats):
        print(f'mse {stats[0]}') 
        print(f'bias {stats[1][:25]}')
        print(f'var {stats[2]}')

    print_some_stats(stats_ridge)
    print_some_stats(stats_theoretical)
    # print(stats_ols[0])
    # print(stats_ols[1][:25])
    # print(stats_ols[2][:25])

# steps to conduct
# fix first exercis - choose lambda through some k-cross validation
# do cross validation - take error from that
# in terms of lambda choice - do some grid search - like from 0 to 1 in steps of 0.01
# fix theoretical values for mse and bias and variance

# repeat in 2b
# how to conduct 2a?? - how to minimize PE??


# not sure if resutls are correct here
task1(size=(1000,950), k=20, repetition=3)

[3.5 3.5 3.5 3.5 3.5 3.5 3.5 3.5 3.5 3.5 3.5 3.5 3.5 3.5 3.5 3.5 3.5 3.5
 3.5 3.5 0.  0.  0.  0.  0. ]
mse [0.21015869 0.21115908 0.21332579]
bias [ 2.95473297  2.84912423  2.7446927   3.03191618  2.91203947  2.82386619
  2.58345401  2.73887884  2.87961662  2.7387423   2.88975543  2.98639937
  2.66661404  2.81734893  2.90258206  2.81068627  2.80243504  2.73831224
  2.67640685  2.77700289  0.1963749  -0.05491973  0.2474     -0.10068103
 -0.3169851 ]
var [1.00081188e-02 2.24557261e-03 3.71050589e-03 2.53761740e-02
 2.36977031e-02 9.35660389e-03 1.49704898e-02 4.45828129e-03
 7.51975706e-05 1.65576748e-02 2.91499415e-02 5.58688696e-02
 3.72159005e-02 2.37179265e-02 5.76798588e-02 4.70959357e-02
 2.56210614e-02 2.12690211e-02 6.72171897e-02 1.66576411e-02
 1.03987111e-02 3.64346958e-02 2.56346556e-02 2.22490971e-02
 3.35461676e-02 4.94945182e-02 7.19742654e-03 3.23955807e-02
 2.11725035e-02 7.59425736e-02 2.48728077e-02 3.06270768e-02
 7.19738443e-02 1.99830209e-03 8.24170207e-03 1.3953072

In [ ]:
# TODO
def task2(size, k=5, sigma=1, repetition=5):

    # beta_true_arr = np.zeros((repetition, size[1]))
    # beta_ridge_arr = np.zeros((repetition, size[1]))
    # beta_ols_arr = np.zeros((repetition, size[1]))

    n_rows, n_cols = size
    for i in range(repetition):

        X, Y, beta, eps = generate_data(orthonormal=False, sigma=1/(np.sqrt(1000)))

        beta = np.zeros((n_cols,1))
        beta[:k] = 3.5



task1(size=(1000,950), k=20, repetition=3)

In [132]:

def check_id_con(Y, X, beta): # taken from lecture, translated to python with chatgpt
    p = X.shape[1]
    
    obj = np.ones(2*p).reshape(2*p, 1)
    constmat1 = np.hstack((X, -X))
    constmat2 = np.hstack((np.eye(p), np.zeros((p, p))))
    constmat3 = np.hstack((np.zeros((p, p)), np.eye(p)))
    constmat = np.vstack((constmat1, constmat2, constmat3))
    # consteq = ['=' for _ in range(Y.shape[0])] + ['>=' for _ in range(2*p)]
    
    constright = np.vstack((Y, np.zeros(2*p).reshape(2*p, 1)))
    # print(constright.shape, constmat.shape)
    
    res = linprog(c=obj, A_ub=(-1)*constmat[Y.shape[0]:], b_ub=(-1)*constright[Y.shape[0]:], 
                A_eq=constmat[:Y.shape[0]:], b_eq=constright[:Y.shape[0]])
    # print(res)
    wyn = res.x[:p] - res.x[p:2*p]
    success = np.max(np.abs(wyn - beta)) < 1E-7
    
    return success


def generate_data_task3(size=(100, 200), non_zero_count=200):
    sigma = np.zeros(size) + 0.7
    for i in range(sigma.shape[0]):
        sigma[i, i] = 1
    # sigma = sigma / size[0]
    # X = np.random.normal(loc=0, scale=1, size=size) #/ size[0]
    X = np.random.normal(loc=0.0, scale=sigma, size=size) / size[0]
    print(X.shape)

    # irrepresentability condition
    for i in range(1, non_zero_count):
        S_beta = np.zeros((i, 1))
        S_beta[:i] = 1

        X_I = X[:, :i]
        X_I_bar = X[:, i:]
        max_k_value = np.max(X_I_bar.T @ X_I @ np.linalg.inv(X_I.T @ X_I) @ S_beta)
        # print(max_k_value)
        if max_k_value > 1:
            print(i, max_k_value) # Here value around 10-20 is okay
            max_k = i
            break

    beta = np.zeros((size[1],1))
    beta[:max_k] = 20
    Y = X @ beta

    # discover all important varaibels using LASSO    
    alphas = np.linspace(0, 0.001, 50)
    for i in alphas:
        if i == 0: continue
        model = Lasso(alpha=i).fit(X, Y)
        # print(model.coef_[:20])
        non_zeros_lasso = np.sum(model.coef_ > 1e-10)
        
        if non_zeros_lasso == max_k:
            print(i, non_zeros_lasso)
            break

    # identifiability condition - for some reason doesn't work properly
    for i in range(1, non_zero_count):
        # print(i)
        beta = np.zeros((size[1],1))
        beta[:non_zero_count] = 20
        Y = X @ beta
        id_condition = check_id_con(Y, X, beta)
        if id_condition:
            print(f'id cond is {non_zero_count}')
            break

for i in range(1):

    generate_data_task3(non_zero_count=100, )

(100, 200)
13 1.0427023737160357
0.0006326530612244898 13
